In [1]:
# import logging
%load_ext autoreload
%autoreload 2

import sys
import os
from collections import Counter
import dotenv

sys.path.append('../') # allow importing of other modules
import rag_gold.rag_pipeline as rpl

# The following loads file with environment variables (such as LLM API Keys)
# change path to file as needed
dotenv.load_dotenv(os.environ['HOME'] + '/profile.env', override=True)

from llama_index.core import SimpleDirectoryReader
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
documents = SimpleDirectoryReader("../rag_docs").load_data()

Counter((doc.metadata['file_path'], doc.metadata.get('file_type'))
        for doc in documents )

Counter({('/home/teo/git/llm_finetunning/rag_gold/../rag_docs/Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.preprocessed.txt',
          'text/plain'): 1,
         ('/home/teo/git/llm_finetunning/rag_gold/../rag_docs/Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.translated.txt',
          'text/plain'): 1,
         ('/home/teo/git/llm_finetunning/rag_gold/../rag_docs/reglamento-maestria-web-2024.preprocessed.txt',
          'text/plain'): 1,
         ('/home/teo/git/llm_finetunning/rag_gold/../rag_docs/reglamento-maestria-web-2024.translated.txt',
          'text/plain'): 1})

In [ ]:
from src.utils import get_secret
from llama_index.llms.anthropic import Anthropic

llm = Anthropic(model= , api_key=get_secret("ANTHROPIC_API_KEY"))
q_engine = rpl.make_query_engine(documents,
                                 chunk_size=700,
                                 chunk_overlap=300,
                                 llm=llm,
                                 similarity_top_k=5)


Returning secret from environment variable `ANTHROPIC_API_KEY`=`sk...AA`
Using LLM=callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fd983fa1a50> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x7fda3b9c9940> completion_to_prompt=<function default_completion_to_prompt at 0x7fda3b8a7ba0> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None model='claude-2.1' temperature=0.1 max_tokens=512 base_url=None timeout=None max_retries=10 additional_kwargs={}
Embedding dimension: 384
chunks: 103
nodes: 103
computing embeddings now...


100%|██████████| 103/103 [00:01<00:00, 55.77it/s]

adding 103 to vector_store


In [4]:
import pandas as pd
test_df = pd.read_csv("../data/test_uniandes.csv")

test_df.iloc[1]

def format_question(row: pd.Series) -> str:
    return """{question}
    Selecciona una sola de las siguientes opciones:
    a. {op1}
    b. {op2}
    c. {op3}
    d. {op4}
    IMPORTANTE: Responde solo con una letra a, b, c, o d, correspondiente a la mejor respuesta. No agregues comentarios ni justificaciones.
    """.format(
        question=row['Pregunta'],
        op1=row['Opcion1'],
        op2=row['Opcion2'],
        op3=row['Opcion3'],
        op4=row['Opcion4'],
    )

print(format_question(test_df.iloc[0]))

¿Cuál es la naturaleza jurídica de la Universidad de los Andes según sus estatutos?
    Selecciona una sola de las siguientes opciones:
    a. Entidad privada con ánimo de lucro
    b. Institución pública
    c. Corporación de utilidad común sin ánimo de lucro
    d. Fundación educativa mixta
    IMPORTANTE: Responde solo con una letra a, b, c, o d, correspondiente a la mejor respuesta. No agregues comentarios ni justificaciones.
    


In [5]:
answers_by_id = {}

In [6]:
from src.utils import get_secret
from llama_index.core.query_engine import RetrieverQueryEngine

# groq_api_keys = get_secret('GROQ_API_KEYS').split(';')
# print('groq_api_keys', len(groq_api_keys))
# llm = rpl.Groq(rpl.GROQ_MODEL, api_key=groq_api_keys[2])

q_engine = RetrieverQueryEngine.from_args(q_engine.retriever, llm=llm)

In [7]:
# Responde solo con un número entre 1 y 4, correspondiente a la respuesta.
import time

q_engine.retriever._verbose = False

for _, row in test_df.iterrows():
    id_ = row['ID']
    if answers_by_id.get(id_) in {'a', 'b', 'c', 'd'}:
        print(f"\nID={id_} already answered")
        continue

    formatted_question = format_question(row)
    # print(id_, formatted_question)
    response = str(q_engine.query(formatted_question))
    time.sleep(1)
    print(f"{id_} - RESPONSE: {response}\n\n")
    answers_by_id[id_] = response


RESPONSE: Unfortunately I do not have enough context in the provided information to definitively state the legal nature of Universidad de los Andes according to its statutes. The provided text does not directly reference or quote from the university's statutes regarding its legal nature. Without more context about what is stated in the statutes, I cannot reliably select one of the multiple choice options. I apologize that I cannot provide a more substantive response based on the limited information given.


RESPONSE: c. En la ciudad de Bogotá


RESPONSE: c


RESPONSE: No hay información en el contexto proporcionado que responda específicamente la pregunta sobre la duración del período inicial para el rector de la Universidad. El contexto habla sobre reglamentos académicos y disciplinarios para estudiantes de maestría, pero no menciona la duración del período del rector. Dado que no puedo responder apropiadamente la pregunta basándome en la información dada, no seleccionaré ninguna de l

In [ ]:
print(str(response))

La respuesta correcta es la opción 3: Corporación de utilidad común sin ánimo de lucro.

Según el texto, la Universidad de los Andes se fundó como "institución de enseñanza superior, de carácter privado, sin ánimo de lucro". Esto indica que la Universidad es una entidad privada que no busca obtener beneficios económicos, sino que se enfoca en la educación y el bien común.


In [ ]:
answers_by_id

{1: 'c',
 2: 'No se proporciona información sobre el domicilio principal de la Universidad de los Andes en el contexto dado. Sin embargo, se menciona que la Universidad de los Andes se fundó para ofrecer a la juventud colombiana nuevas oportunidades de educación, lo que sugiere que la Universidad se encuentra en Colombia. Por lo tanto, la respuesta más adecuada sería:\n\nd',
 3: 'c',
 4: 'No hay información disponible en el contexto proporcionado sobre la duración del período inicial para el rector de la Universidad.',
 5: 'No se proporciona información sobre el número máximo de consejeros ordinarios que puede tener el Consejo Superior en el texto dado.',
 6: 'c',
 7: 'c',
 8: 'c',
 9: 'b',
 10: 'd',
 11: 'No hay información disponible en el contexto proporcionado sobre el período de nombramiento para el revisor fiscal de la Universidad.',
 12: 'd',
 13: 'No se proporciona información sobre la frecuencia de reuniones del Consejo Superior en el texto proporcionado. Sin embargo, se puede

In [11]:
df = pd.Series(answers_by_id).reset_index()
df.columns= ['ID', 'Claude-2.1-Answer-ch700']
df.to_csv("../data/rag-answers-claude-ch700-2.1-DONT-SUBMIT.csv", index=False)
df

,ID,Claude-2.1-Answer-ch700
0,1,Unfortunately I do not have enough context in ...
1,2,c. En la ciudad de Bogotá
2,3,c
3,4,No hay información en el contexto proporcionad...
4,5,No hay suficiente información en el contexto p...
...,...,...
95,96,d. 3.0\n\nLa respuesta correcta es d. 3.0\n\nS...
96,97,b) 35%
97,98,b. Título de Especialista
98,99,c
